In [3]:
#this is the base code, transparent background is either black or white and in the end alll shirts are opened
import cv2
import tkinter as tk
import os
import numpy as np
import time

prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

cam = cv2.VideoCapture(1)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 520)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 380)
cam.set(cv2.CAP_PROP_FPS, 30)

dress_images = []
current_dress_index = 0
user_picture_path = r'.\assets'
output_image_paths = []

def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(12):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            processed_img = process_dress_image(img)
            if processed_img is not None:
                dress_images.append(processed_img)
            else:
                print(f'Error processing image: {image_path}')
        else:
            print(f'Error loading image: {image_path}')

def process_dress_image(img):
    try:
        if img.shape[2] == 4:
            return img
        else:
            b, g, r = cv2.split(img)
            alpha = np.ones_like(b) * 255
            rgba_img = cv2.merge((b, g, r, alpha))
            return rgba_img
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

def apply_dress(frame, dress_index, detections):
    global dress_images
    if dress_images and dress_index < len(dress_images):
        confidence_threshold = 0.5  
        detected_faces = []
        
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > confidence_threshold:
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                detected_faces.append((x, y, w, h))

        if detected_faces:
            dress_img = dress_images[dress_index]
            aspect_ratio = dress_img.shape[1] / dress_img.shape[0]

            for (x, y, w, h) in detected_faces:
                y_dress = y + int(0.8 * h)
                h_dress = int(0.5 * h)
                x_offset = int(0.5 * w)
                x_dress = max(x - x_offset, 0)
                w_dress = min(w + 2 * x_offset, frame.shape[1] - x_dress)
                new_h_dress = int(w_dress / aspect_ratio)

                if new_h_dress > 0:
                    resized_dress = cv2.resize(dress_img, (w_dress, new_h_dress), interpolation=cv2.INTER_LINEAR)

                    if resized_dress.shape[2] == 4:
                        resized_dress = resized_dress[:, :, :3]

                    frame[y_dress:y_dress + new_h_dress, x_dress:x_dress + w_dress] = resized_dress[
                                                                                        :min(new_h_dress,
                                                                                             frame.shape[0] - y_dress),
                                                                                        :min(w_dress,
                                                                                             frame.shape[1] - x_dress)]
                    
def process_and_save_output(frame, dress_index, detections):
    global user_picture_path, dress_images, output_image_paths
    if dress_images and user_picture_path:
        output_folder = 'outputs'
        os.makedirs(output_folder, exist_ok=True)

        output_path = os.path.join(output_folder, f'output{dress_index}.png')  
        frame_copy = frame.copy()
        apply_dress(frame_copy, dress_index, detections)
        cv2.imwrite(output_path, frame_copy)
        print(f'Saved output image: {output_path}')

        output_image_paths.append(output_path)

def main():
    select_dress_images()
    global current_dress_index

    start_time = time.time()
    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        apply_dress(frame, current_dress_index, detections)
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break
        elif key == 13:
            process_and_save_output(frame, current_dress_index, detections)
            current_dress_index = (current_dress_index + 1) % len(dress_images)
            if current_dress_index == 0: 
                break
        
        if time.time() - start_time >= 3:
            process_and_save_output(frame, current_dress_index, detections)
            current_dress_index = (current_dress_index + 1) % len(dress_images)
            if current_dress_index == 0: 
                break
            start_time = time.time()

    cv2.destroyAllWindows()
    cam.release()

    # Open all saved images
    for image_path in output_image_paths:
        os.system(image_path)
        
main()


In [ ]:
# same as above,  transparent background is either black or white and in the end alll shirts are opened
import cv2
import numpy as np
import tkinter as tk
import time
import os

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)


# Initialize webcam
cam = cv2.VideoCapture(1)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)


dress_images = []
current_dress_index = 0
user_picture_path = r'.\assets'
output_image_paths = []


def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(12):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            processed_img = process_dress_image(img)
            if processed_img is not None:
                dress_images.append(processed_img)
            else:
                print(f'Error processing image: {image_path}')
        else:
            print(f'Error loading image: {image_path}')



def process_dress_image(img, alpha_value=255):
    try:
        if img.shape[2] == 4:
            return img  # Image already has an alpha channel
        else:
            b, g, r = cv2.split(img)
            alpha = np.ones_like(b) * alpha_value  # Adjust alpha value here
            rgba_img = cv2.merge((b, g, r, alpha))
            return rgba_img
    except Exception as e:
        print(f"Error processing image: {e}")
        return None




def apply_dress(frame, dress_index, detections):
    global dress_images
    if dress_images and dress_index < len(dress_images):
        confidence_threshold = 0.5  
        detected_faces = []
        
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > confidence_threshold:
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                detected_faces.append((x, y, w, h))

        if detected_faces:
            dress_img = dress_images[dress_index]
            aspect_ratio = dress_img.shape[1] / dress_img.shape[0]

            for (x, y, w, h) in detected_faces:
                y_dress = y + int(0.8 * h)
                h_dress = int(0.5 * h)
                x_offset = int(0.5 * w)
                x_dress = max(x - x_offset, 0)
                w_dress = min(w + 2 * x_offset, frame.shape[1] - x_dress)
                new_h_dress = int(w_dress / aspect_ratio)

                if new_h_dress > 0:
                    resized_dress = cv2.resize(dress_img, (w_dress, new_h_dress), interpolation=cv2.INTER_LINEAR)

                    if resized_dress.shape[2] == 4:
                        resized_dress = resized_dress[:, :, :3]

                    frame[y_dress:y_dress + new_h_dress, x_dress:x_dress + w_dress] = resized_dress[
                                                                                        :min(new_h_dress,
                                                                                             frame.shape[0] - y_dress),
                                                                                        :min(w_dress,
                                                                                             frame.shape[1] - x_dress)]
                    

def process_and_save_output(frame, dress_index, detections):
    global user_picture_path, dress_images, output_image_paths
    if dress_images and user_picture_path:
        output_folder = 'outputs'
        os.makedirs(output_folder, exist_ok=True)

        output_path = os.path.join(output_folder, f'output{dress_index}.png')  
        frame_copy = frame.copy()
        apply_dress(frame_copy, dress_index, detections)
        cv2.imwrite(output_path, frame_copy)
        print(f'Saved output image: {output_path}')

        output_image_paths.append(output_path)


def main():
    select_dress_images()
    global current_dress_index

    time.sleep(0) #later change the digit to 5 # 5 because it gives us time settlte ourselves
    
    start_time = time.time()
    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        apply_dress(frame, current_dress_index, detections)
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break
        elif key == 13: #press enter to get next shirt
            process_and_save_output(frame, current_dress_index, detections)
            current_dress_index = (current_dress_index + 1) % len(dress_images)
            if current_dress_index == 0: 
                break
        
        if time.time() - start_time >= 3:
            process_and_save_output(frame, current_dress_index, detections)
            current_dress_index = (current_dress_index + 1) % len(dress_images)
            if current_dress_index == 0: 
                break
            start_time = time.time()

    cv2.destroyAllWindows()
    cam.release()

    # Open all saved images in the end
    for image_path in output_image_paths:
        os.system(image_path)
        
main()

# transparency

In [1]:
#this code works as transparent but the shirt is being applied above the face
#but working with the laptop camera 
#middle values in the error indicates our face x,y locations
# this saves the picture and  then applies the shirts

import cv2
import os
import numpy as np
import time

prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 520)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 380)
cam.set(cv2.CAP_PROP_FPS, 30)

dress_images = []
user_picture_path = r'.\assets'
output_folder = 'outputs'
os.makedirs(output_folder, exist_ok=True)

def process_dress_image(img):
    try:
        if img.shape[2] == 4:
            return img  # Image already has an alpha channel
        else:
            b, g, r = cv2.split(img)
            alpha = np.ones_like(b) * 255
            rgba_img = cv2.merge((b, g, r, alpha))
            return rgba_img
    except Exception as e:
        print(f"Error processing image: {e}")
        return None
    
def take_and_save_picture():
    ret, frame = cam.read()
    if not ret:
        print('Error: Failed to open webcam or read frame')
        return None
    output_path = os.path.join(output_folder, 'captured_image.png')
    cv2.imwrite(output_path, frame)
    print(f'Saved captured image: {output_path}')
    return output_path

def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(12):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            processed_img = process_dress_image(img)
            if processed_img is not None:
                dress_images.append(processed_img)
            else:
                print(f'Error processing image: {image_path}')
        else:
            print(f'Error loading image: {image_path}')

def apply_dress_to_picture(image_path):
    global dress_images
    if not dress_images:
        print('Error: No dress images found')
        return
    
    img = cv2.imread(image_path)
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
            (x, y, w, h) = box.astype(int)
            dress_img = dress_images[i % len(dress_images)]  # Choose a dress from the available options
            resized_dress = cv2.resize(dress_img, (w, h))

            # Perform alpha blending
            alpha_mask = resized_dress[:, :, 3] / 255.0
            alpha_mask = np.repeat(alpha_mask[:, :, np.newaxis], 3, axis=2)  # Convert to 3-channel alpha mask
            blended_img = img.copy()
            blended_img[y:y+h, x:x+w] = alpha_mask * resized_dress[:, :, :3] + (1 - alpha_mask) * img[y:y+h, x:x+w]
            img = blended_img
    
    output_path = os.path.join(output_folder, 'output_image.png')
    cv2.imwrite(output_path, img)
    print(f'Saved output image with dresses: {output_path}')


def main():
    select_dress_images()
    picture_path = take_and_save_picture()
    if picture_path:
        apply_dress_to_picture(picture_path)
    cv2.destroyAllWindows()
    cam.release()

main()


Error loading image: .\assets\shirt11.png
Saved captured image: outputs\captured_image.png


ValueError: operands could not be broadcast together with shapes (145,400,3) (0,387,3) 

In [2]:
#this code works as transparent but the shirt is being applied above the face
#but working with the laptop camera 
#middle values in the error indicates our face x,y locations
# this saves the picture and  then applies the shirts

import cv2
import os
import numpy as np
import time

prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 520)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 380)
cam.set(cv2.CAP_PROP_FPS, 30)

dress_images = []
user_picture_path = r'.\assets'
output_folder = 'outputs'
os.makedirs(output_folder, exist_ok=True)

def process_dress_image(img):
    try:
        if img.shape[2] == 4:
            return img  # Image already has an alpha channel
        else:
            b, g, r = cv2.split(img)
            alpha = np.ones_like(b) * 255
            rgba_img = cv2.merge((b, g, r, alpha))
            return rgba_img
    except Exception as e:
        print(f"Error processing image: {e}")
        return None
    
def take_and_save_picture():
    ret, frame = cam.read()
    if not ret:
        print('Error: Failed to open webcam or read frame')
        return None
    output_path = os.path.join(output_folder, 'captured_image.png')
    cv2.imwrite(output_path, frame)
    print(f'Saved captured image: {output_path}')
    return output_path

#this code works as transparent but the shirt is being applied above the face
#but working with the laptop camera 
#middle values in the error indicates our face x,y locations
# this saves the picture and  then applies the shirts

import cv2
import os
import numpy as np
import time

prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 520)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 380)
cam.set(cv2.CAP_PROP_FPS, 30)

dress_images = []
user_picture_path = r'.\assets'
output_folder = 'outputs'
os.makedirs(output_folder, exist_ok=True)

def process_dress_image(img):
    try:
        if img.shape[2] == 4:
            return img  # Image already has an alpha channel
        else:
            b, g, r = cv2.split(img)
            alpha = np.ones_like(b) * 255
            rgba_img = cv2.merge((b, g, r, alpha))
            return rgba_img
    except Exception as e:
        print(f"Error processing image: {e}")
        return None
    
def take_and_save_picture():
    ret, frame = cam.read()
    if not ret:
        print('Error: Failed to open webcam or read frame')
        return None
    output_path = os.path.join(output_folder, 'captured_image.png')
    cv2.imwrite(output_path, frame)
    print(f'Saved captured image: {output_path}')
    return output_path

def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(12):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            processed_img = process_dress_image(img)
            if processed_img is not None:
                dress_images.append(processed_img)
            else:
                print(f'Error processing image: {image_path}')
        else:
            print(f'Error loading image: {image_path}')

def apply_dress_to_picture(image_path):
    global dress_images
    if not dress_images:
        print('Error: No dress images found')
        return
    
    img = cv2.imread(image_path)
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
            (x, y, w, h) = box.astype(int)
            
            # Adjusting y-coordinate to position the dress a bit below
            y_offset = int(0.4 * h)  # You can adjust this value as needed
            y += y_offset

            dress_img = dress_images[i % len(dress_images)]  # Choose a dress from the available options
            resized_dress = cv2.resize(dress_img, (w, h))

            # Perform alpha blending
            alpha_mask = resized_dress[:, :, 3] / 255.0
            alpha_mask = np.repeat(alpha_mask[:, :, np.newaxis], 3, axis=2)  # Convert to 3-channel alpha mask
            blended_img = img.copy()
            blended_img[y:y+h, x:x+w] = alpha_mask * resized_dress[:, :, :3] + (1 - alpha_mask) * img[y:y+h, x:x+w]
            img = blended_img
    
    output_path = os.path.join(output_folder, 'output_image.png')
    cv2.imwrite(output_path, img)
    print(f'Saved output image with dresses: {output_path}')


def main():
    select_dress_images()
    picture_path = take_and_save_picture()
    if picture_path:
        apply_dress_to_picture(picture_path)
    cv2.destroyAllWindows()
    cam.release()

main()


Error loading image: .\assets\shirt11.png
Saved captured image: outputs\captured_image.png


ValueError: operands could not be broadcast together with shapes (144,399,3) (144,388,3) 